In [1]:
import sys
sys.path.append('D:/UNESP/EngMec-UNESP/')
import frankenstein as fk

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import ZebraLib as zb
import uncertainties.unumpy as un

D1=0.0385; A1 = np.pi*(D1**2)/4
Dt=0.0257; At = np.pi*(Dt**2)/4
beta = Dt/D1

def f_distrib(hl, Vm):
    D1=0.0385 #m
    L = 5     #m
    return (2*D1*hl)/(L*Vm**2)

def f_blassius(Re):
    return 0.316/(Re**0.25)

def calc_rugosidade(Re):
    def rugosidade(Rug, args):
        f = f_blassius(Re)

        A = ((Rug/3.7)**1.11) + (6.9/Re)
        return 1/np.sqrt(f) + 1.8*np.log10(A)
    Rug, it, res = zb.robustNewton(rugosidade, np.array([3e-4]), args=Re)
    return Rug

def hl(delta_h):
    '''
    Calcula a perda de carga hl dado  
    diferença de H do tubo em U [cm] com água.
    '''
    delta_P = fk.rho_agua*fk.g*(delta_h/100)
    return delta_P/fk.rho_ar

def calc_vazão(delta_h):
    '''
    Calcula a vazão em massa, número de Reynouds e velocidade. Obs: delta_h [cm]
    '''
    D1=0.0385; A1 = np.pi*(D1**2)/4
    Dt=0.0257; At = np.pi*(Dt**2)/4
    beta = Dt/D1

    def residuo_Re(Re, delta_h):
        delta_P = fk.rho_agua*fk.g*(delta_h/100)

        C = 0.5959 + (0.0312*(beta**2.1)) - (0.184*beta**8) + ((91.71*beta**2.5)/(Re**0.75))
        K = C/np.sqrt(1 - beta**4)
        return ((fk.rho_ar*Re*fk.mi*A1)/D1) - (K*At*np.sqrt(2*fk.rho_ar*delta_P))
        
    Re, it, res = (zb.robustNewton(residuo_Re, np.array([1e6]), args=delta_h))
    
    dot_m = (fk.rho_ar*Re*fk.mi*A1)/D1
    velocidade = (Re*fk.mi)/D1
    return float(dot_m), float(Re), float(velocidade) #kg/s, adim, m/s

def vazão_from_dataframe(df_, key=0):
    '''
    Calcula a vazão em massa, número de Reynouds e velocidade e retorna em um Dataframe. Obs: delta_h [cm]
    '''
    arr_po = np.zeros((df_.shape[0], df_.shape[1]-2))
    for j in range(arr_po.shape[1]):
        col_nome = ['PO_TL', 'PO_TR'][j]
        for i in range(arr_po.shape[0]):
            delta_h = df_[col_nome].values[i]
            arr_po[i, j] = calc_vazão(delta_h)[key] #[0]kg/s, [1]adim, [2]m/s
            
    return pd.DataFrame(arr_po, columns=['PO_TL', 'PO_TR'])

df = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 3 - Perda de carga/dados3.xlsx', 
                    sheet_name='df_data')

df3 = df[0:4]; df5 = df[4:8]; df7 = df[8:12]; dfa = df[12:16]
lista_df = [df3, df5, df7, dfa]
df3
df

,PO_TL,TL,PO_TR,TR
0,6.4,1.9,2.3,2.7
1,6.0,2.0,2.5,1.5
2,6.4,1.9,2.2,2.3
3,6.4,1.8,2.2,2.4
4,13.6,3.6,6.5,7.2
5,13.2,3.5,6.5,7.1
6,12.9,3.4,5.9,6.9
7,12.5,3.4,6.2,6.7
8,17.6,4.3,8.0,8.9
9,17.2,4.4,8.0,9.0


In [2]:
print(df.to_latex(index=0))

\begin{tabular}{rrrr}
\toprule
 PO\_TL &  TL &  PO\_TR &  TR \\
\midrule
   6.4 & 1.9 &    2.3 & 2.7 \\
   6.0 & 2.0 &    2.5 & 1.5 \\
   6.4 & 1.9 &    2.2 & 2.3 \\
   6.4 & 1.8 &    2.2 & 2.4 \\
  13.6 & 3.6 &    6.5 & 7.2 \\
  13.2 & 3.5 &    6.5 & 7.1 \\
  12.9 & 3.4 &    5.9 & 6.9 \\
  12.5 & 3.4 &    6.2 & 6.7 \\
  17.6 & 4.3 &    8.0 & 8.9 \\
  17.2 & 4.4 &    8.0 & 9.0 \\
  17.2 & 4.4 &    7.9 & 8.8 \\
  17.2 & 4.3 &    7.8 & 8.9 \\
  18.7 & 4.5 &    8.6 & 9.7 \\
  18.7 & 4.6 &    8.8 & 9.7 \\
  18.6 & 4.6 &    8.6 & 9.7 \\
  18.7 & 4.5 &    8.6 & 9.8 \\
\bottomrule
\end{tabular}



In [138]:
df_ = df; key=1

arr_po = np.zeros((df_.shape[0], df_.shape[1]))
for j in range(arr_po.shape[1]):
    col_nome = ['PO_TL', 'PO_TR', 'TL', 'TR'][j]
    fun = [calc_vazão, calc_vazão, hl, hl]
    for i in range(arr_po.shape[0]):

        delta_h = df_[col_nome].values[i]
        try:
            value = fun[j](delta_h)[key]
        except:
            value = fun[j](delta_h)

        arr_po[i, j] = value #[0]kg/s, [1]adim, [2]m/s
        
df_51 = pd.DataFrame(arr_po, columns=['key_TL', 'key_TR', 'h_TL', 'h_TR'])
df_51

,key_TL,key_TR,h_TL,h_TR
0,21350.889278,12975.858923,154.859025,220.062825
1,20687.975554,13510.657510,163.009500,122.257125
2,21350.889278,12699.647786,154.859025,187.460925
3,21350.889278,12699.647786,146.708550,195.611400
4,30894.658717,21513.333229,293.417100,586.834200
5,30444.762546,21513.333229,285.266625,578.683725
6,30102.827493,20518.799056,277.116150,562.382775
7,29640.636968,21022.123450,277.116150,546.081825
8,35071.422566,23813.883712,350.470425,725.392275
9,34676.851811,23813.883712,358.620900,733.542750


In [184]:
it = [[0, 4], [4, 8], [8, 12], [12, 15]]
nominal_values, std_devs = [], []

unarray = np.zeros((4, 4))

for j in range(4):
    col_nome = df_.columns[j]
    print(col_nome)
    for i in range(4):

        amostra = df_[col_nome][it[j][0]:it[j][1]].values
        média = amostra.mean()
        incert = amostra.std(ddof=1)/np.sqrt(len(amostra))
        print(f'{média} -- {incert}')
    print('----')
        #unarray[i, j] = ufloat(média, incert)

PO_TL
6.3 -- 0.09999999999999995
6.3 -- 0.09999999999999995
6.3 -- 0.09999999999999995
6.3 -- 0.09999999999999995
----
TL
3.4749999999999988 -- 0.04787135538781681
3.4749999999999988 -- 0.04787135538781681
3.4749999999999988 -- 0.04787135538781681
3.4749999999999988 -- 0.04787135538781681
----
PO_TR
7.924999999999999 -- 0.04787135538781712
7.924999999999999 -- 0.04787135538781712
7.924999999999999 -- 0.04787135538781712
7.924999999999999 -- 0.04787135538781712
----
TR
9.7 -- 0.0
9.7 -- 0.0
9.7 -- 0.0
9.7 -- 0.0
----


In [137]:
def df_incert_grupo(df):
    it = [[0, 4], [4, 8], [8, 12], [12, 16]]
    nominal_values, std_devs = [], []
    for i in it:
        values = df[i[0]:i[1]].values
        nominal_values.append(values.mean())
        std_devs.append(values.std(ddof=1)/np.sqrt(len(values)))
    return un.uarray(nominal_values, std_devs) 

df_incert_grupo(df_51)

array([8623.20523551383+/-4617.944675958123,
       13066.085827389445+/-6738.89468400593,
       14889.853685603961+/-7681.487471000229,
       15503.984933041993+/-7980.852243657584], dtype=object)